# In-class notebook: 2024-01-08

In this notebook, we will re-familiarize ourselves with the basics of probability.

This notebook is intended to support Chapter 3.1 of the textbook, and material is taken from the following scripts (from astroML):
* https://github.com/astroML/astroML-notebooks/blob/main/chapter3/astroml_chapter3_Overview_of_Probability_and_Random_Variables.ipynb

## Probability axioms

Given an event $A$, such as the outcome of a coin toss, we assign it a real number $p(A)$, called the probability of $A$. Note that $p(A)$ could also correspond to a probability that a value of $x$ falls in a $dx$ wide interval around $x$. To qualify as a probability, $p(A)$ must satisfy three Kolmogorov axioms:

1. $p(A) \geq 0$ for each $A$.
2. $p(\Omega) = 1$, where $\Omega$ is the set of all possible outcomes.
3. If $A_1$, $A_2$, . . . are disjoint events, then $p (\bigcup^{\infty}_{i=1}A_i) = \sum_{i=1}^{\infty}p(A_i)$ where $\bigcup$ stands for “union.”

### Several useful rules can be derived as a consequence of these axioms

**Sum rule**:  The probability that the union of two events, $A$ and $B$, will happen is given by,

$$\qquad\qquad p(A \cup B) = p(A) + p(B) - p(A \cap B)\qquad\qquad\qquad(1)$$


which is the sum of $A$ and $B$'s respective probabilities minus the probability that both $A$ and $B$ will happen. The union of two events is the probability that *either* event occurs. The $\cap$ in the equation stands for "intersection", and subtracting the last term, $p(A \cap B)$, avoids double counting the places that $A$ and $B$ overlap. 

**Rule of complementary events**: The probability of an event happening plus the probability of it not happening is equal to 1.
$$ p(A) + p(\overline{A}) = 1 $$

**Rule of multiplication**: The probability that both events $A$ and $B$ occur is equal to the probability that event $B$ occurs times the probability that event $A$ occurs given that $B$ has occurred. Switching $A$ and $B$ yields an equivalent statement.

$$ p(A \cap B) = p(A|B)p(B) = p(B|A)p(A)$$

In the equation above, "|" is pronounced "given," and $p(A|B)$ is the probability of event $A$ occurring given that B has occurred.

**Example**: Find the probability of rolling an odd number greater than 2.

> Event A: Rolling an odd number
>
> Event B: Rolling a number greater than 2
>
>$P(\text{odd and greater than 2}) = P(\text{odd|if} >2) \times P(> 2)$
>    
>- All outcomes: 1 2 3 4 5 6
>- $P(\text{is}$ > 2) = count(3 4 5 6) / count (1 2 3 4 5 6) = 4/6
>- the probability that a number is odd if it's greater than two:  3,5
>- $P(\text{is odd| if} > 2)$ = count(3 5) / count (3 4 5 6) = 1/2
>   
> $P(\text{odd and greater than 2})$ = 4/6 $\times$ 1/2 = $\fbox{1/3}$
>   
>We can get the same answer by determining the odd numbers greater than two on a die (2) and dividing by the total possibilities (6) to get 1/3.


```{toggle}
XXX
```